In [1]:
# Setup the Jupyter version of Dash
from dash import Dash


# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output
import base64
import os
import pandas as pd
import matplotlib.pyplot as plt

from crud import AnimalShelter
import socket
from dash import callback_context
###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "JL033124"
mongo_host = "nv-desktop-services.apporto.com"
mongo_port = 31955
db = "AAC"
collection = "animals"

shelter = AnimalShelter(username, password, mongo_host, mongo_port, db, collection)
test_df = pd.DataFrame.from_records(shelter.read({}))
print("Sample data:\n", test_df.head())

df = pd.DataFrame.from_records(shelter.read({}))

if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

rescue_types = ['Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking', 'Reset']

# Initialize the app
app = Dash(__name__)

# Load logo image
image_filename = 'Grazioso Salvare Logo.png'
if os.path.exists(image_filename):
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
    logo_img = html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
else:
    logo_img = html.Div("Logo not found")

# Layout
app.layout = html.Div([
    html.Center(logo_img),
    html.Center(html.B(html.H1('CS-340 Animal Shelter Dashboard- Junyu Liang'))),
    html.Hr(),
    html.Center(html.H4('2025')),
    html.Div(className='buttonRow', style={'display': 'flex'}, children=[
        html.Button('Water Rescue', n_clicks=0, id='submit-button-one'),
        html.Button('Mountain Rescue', n_clicks=0, id='submit-button-two'),
        html.Button('Disaster Rescue', n_clicks=0, id='submit-button-three'),
        html.Button('Reset', n_clicks=0, id='submit-button-four'),
    ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable='single',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=5,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Callback for table filter buttons

@app.callback(
    Output('datatable-id', "data"),
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks'),
     Input('submit-button-three', 'n_clicks'),
     Input('submit-button-four', 'n_clicks')]
)
def on_click(button1, button2, button3, button4):
    ctx = callback_context
    if not ctx.triggered:
        return df.to_dict('records')

    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if triggered_id == 'submit-button-one':
        water_query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lt": 156}
        }
        dff = pd.DataFrame.from_records(shelter.read(water_query))

    elif triggered_id == 'submit-button-two':
        mountain_query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lt": 156}
        }
        dff = pd.DataFrame.from_records(shelter.read(mountain_query))

    elif triggered_id == 'submit-button-three':
        disaster_query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lt": 156}
        }
        dff = pd.DataFrame.from_records(shelter.read(disaster_query))

    else:  # Reset button or fallback
        dff = pd.DataFrame.from_records(shelter.read({}))

    return dff.to_dict('records')


# Graph callback
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if not viewData:
        return [html.P("No data to display.")]

    dff = pd.DataFrame.from_dict(viewData)

    if 'breed' not in dff.columns or dff.empty:
        return [html.P("Breed data is missing or empty.")]

    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Preferred Animals')
        )
    ]

# Style selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{'if': {'column_id': i}, 'background_color': '#E7DED9'} for i in selected_columns or []]

# Map callback
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, selected_rows):
    if not viewData or not selected_rows:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = selected_rows[0]

    lat = dff.iloc[row]['location_lat']
    lon = dff.iloc[row]['location_long']
    breed = dff.iloc[row]['breed']
    name = dff.iloc[row]['name']

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id='base-layer-id'),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([html.H1("Animal Name"), html.P(name)])
            ])
        ])
    ]

# Run the app

def find_open_port(start=8050, end=8100):
    for port in range(start, end):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('localhost', port)) != 0:
                return port
    raise RuntimeError("No available ports")

port = find_open_port()
app.run_server(debug=True, port=port)





Connection to MongoDB successful!
Query returned 19984 documents.
Sample data:
                         _id  rec_num age_upon_outcome animal_id animal_type  \
0  67d8368d36911eb6fc261ae1      3.0          2 years   A716330         Dog   
1  67d8368d36911eb6fc261ae2      1.0          3 years   A746874         Cat   
2  67d8368d36911eb6fc261ae3      9.0          3 years   A720214         Dog   
3  67d8368d36911eb6fc261ae4     10.0         3 months   A664290         Cat   
4  67d8368d36911eb6fc261ae5     11.0           1 year   A721199         Dog   

                     breed        color date_of_birth             datetime  \
0  Chihuahua Shorthair Mix  Brown/White    2013-11-18  2015-12-28 18:43:00   
1   Domestic Shorthair Mix  Black/White    2014-04-10  2017-04-11 09:00:00   
2   Labrador Retriever Mix    Red/White    2013-02-04  2016-02-11 12:41:00   
3   Domestic Shorthair Mix       Tortie    2013-09-01  2013-12-08 14:58:00   
4   Dachshund Wirehair Mix    Tan/White    2015-02-23  